In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"CUDA Device Count: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
   print(f"Device {i}: {torch.cuda.get_device_name(i)}")

Using device: cuda
CUDA Device Count: 1
Device 0: Tesla V100-PCIE-32GB


In [3]:
import json
from tqdm import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.10/dist-packages/torchvision/datapoints/__init__.py:14: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/__init__.py:64: UserWarning: The

In [3]:
model_path = "./t5-finetuned-squad-custom"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path).to(device)

In [4]:
model_name = "google-t5/t5-base"
proxy = "http://sisproxy.hkg.agoda.local:3128"
tokenizer = T5Tokenizer.from_pretrained(model_name, proxies={"http": proxy, "https": proxy})
print('Tokenizer loaded successfully')

model = T5ForConditionalGeneration.from_pretrained(model_name, proxies={"http": proxy, "https": proxy}).to(device)
print('Model loaded successfully')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Tokenizer loaded successfully
Model loaded successfully


In [6]:
def load_squad(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        squad_dict = json.load(f)
    
    dataset = []
    for article in squad_dict["data"]:
        for paragraph in article["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                question = qa["question"]
                qid = qa["id"]
                dataset.append({"id": qid, "context": context, "question": question})
    
    return dataset

dataset = load_squad("dev-v1.1.json")
print('Dataset loaded successfully')

def generate_answer(context, question):
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).input_ids.to(device)
    outputs = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

predictions = {}
for example in tqdm(dataset):
    question = example['question']
    context = example['context']
    qid = example['id']
    
    answer = generate_answer(context, question)
    predictions[qid] = answer

with open('predictions.json', 'w') as f:
    json.dump(predictions, f)
print("Predictions saved to predictions.json")

Dataset loaded successfully


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10570/10570 [18:48<00:00,  9.36it/s]

Predictions saved to predictions.json


In [7]:
!python evaluate-v2.0.py dev-v1.1.json predictions.json

{
  "exact": 80.54872280037843,
  "f1": 89.39400002730301,
  "total": 10570,
  "HasAns_exact": 80.54872280037843,
  "HasAns_f1": 89.39400002730301,
  "HasAns_total": 10570
}


In [ ]:
import json
from tqdm import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration
model_name = "google-t5/t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
print('Tokenizer loaded successfully')

model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
print('Model loaded successfully')
def load_squad(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        squad_dict = json.load(f)
    
    dataset = []
    for article in squad_dict["data"]:
        for paragraph in article["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                question = qa["question"]
                qid = qa["id"]
                dataset.append({"id": qid, "context": context, "question": question})
    
    return dataset

dataset = load_squad("dev-v1.1.json")
print('Dataset loaded successfully')

def generate_answer(context, question):
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).input_ids.to(device)
    outputs = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

predictions = {}
for example in tqdm(dataset):
    question = example['question']
    context = example['context']
    qid = example['id']
    
    answer = generate_answer(context, question)
    predictions[qid] = answer

with open('predictions.json', 'w') as f:
    json.dump(predictions, f)
print("Predictions saved to predictions.json")
